Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [91]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [92]:
PROBLEM_SIZE = 20
NUM_SETS = 12
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [93]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [94]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [95]:
def priority(state):
    return len(state.taken)

def distance(state):    #H function considering both the distance from start and to end -> never overestimate
    return len(state.taken) + PROBLEM_SIZE - sum(
        reduce( np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]), ))


# function to take actions in order -> i want sets that have a lot of valuable tiles 
#   -> compute sum over columns in which it is True / sum over row
order = []
for i in range(NUM_SETS):
    value = 0
    for col in range(PROBLEM_SIZE):
        for j in range(NUM_SETS):    
            value += SETS[j][col]
    value /= SETS[i].sum()
    order.append(value)

def tileSetOrder(n):
    return order[n]


In [96]:
# frontier = LifoQueue()
frontier = PriorityQueue()
# priority queue uses as priority the first element of the tuple -> i will use a custom priority

s = State(set(), set(range(NUM_SETS)))

#consider: if a set has an 'exclusive' tile, we automatically take it
for i in range(PROBLEM_SIZE):
    n = 0
    index = 0
    for j in range(NUM_SETS):
        if SETS[j][i] == True:
            n += 1
            if n >= 2:
                break
            index = j
    if n == 1:
        #print('found exclusive in set:', index, 'at i:', i)        
        s = State(s.taken | {index}, s.not_taken - {index})

#initialize and go
frontier.put((distance(s), s))
counter = 0
current_state = frontier.get()[1]
while not goal_check(current_state):
    counter += 1
    # i want to take actions from the not taken set in a certain order, the most 'valuable' first
    actions = list(current_state.not_taken)
    actions.sort(key = tileSetOrder)
    for action in actions:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )
        frontier.put((distance(new_state), new_state))
    current_state = frontier.get()[1]

print(f"Solved in {counter} steps")

found exclusive in set: 3 at i: 17
Solved in 3 steps


In [97]:
print(current_state)
len(current_state.taken)

State(taken={9, 2, 3, 5}, not_taken={0, 1, 4, 6, 7, 8, 10, 11})


4

In [98]:
goal_check(current_state)

True